In [ ]:
### preprocessing procedures are in two steps
# 1. pick a random non-target images (not clean images)
# 2. split into train, val and test

import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
import shutil
import random
import argparse
import os, shutil
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = os.getcwd()
img_dir = os.path.join(data_dir, 'jpg/')
excel_path = os.path.join(data_dir, 'data/hh_label_all.xlsx')
label_path = os.path.join(data_dir, 'data/hh_label_all.csv') 
df_ex = pd.read_excel(excel_path)

anesthesia_dir = '/data/handhygiene/images/anesthesia/'

In [ ]:
def pick_random_frames():
    for row in df_ex[['video_name', 'anesthesia', 'anesthesia/handhygiene']].values:
        vname = row[0]
        target = row[2]
        if target is np.nan:
            continue
        n_target = len(target.strip().split(','))
        print("target has {} clip/clips. pick a random {} case from video_id {}.".format(n_target, n_target, vname))
        
        for n in range(n_target):
            list = row[1].strip().split(',')
            pick = random.choice(list)
            start, end = pick.split('-')
            l_rand = random.randint(16, 30)
            n_rand = random.randint(int(start), int(end)-l_rand)
            
            new_dir = ''
            for i in range(l_rand):
                dir_name = vname + '_frames' + str(n_rand+i).zfill(6)
                img_name = dir_name + '.jpg'
                if i == 0:
                    if not os.path.exists(os.path.join(anesthesia_dir, 'notclean')):
                        os.mkdir(os.path.join(anesthesia_dir, 'notclean'))
                    new_dir = os.path.join(anesthesia_dir, 'notclean/'+dir_name)
                    os.mkdir(new_dir)
                if new_dir is not '':
                    shutil.copy(os.path.join(anesthesia_dir, img_name), new_dir)

In [ ]:
pick_random_frames()

In [ ]:
test_size= 0.2
val_size = 0.2

In [ ]:
classes = os.listdir(img_dir) # ['notclean', 'clean']
vids = []
for c in classes:
    class_dir = os.path.join(img_dir, c)
    v_name_dirs = os.listdir(class_dir)
    for vname in v_name_dirs:
        vid = vname.split('_')[0]
        if vid not in vids:
            vids.append(vid)

In [ ]:
def split_train_val_test(img_dir, vids):
    x_train, x_test = train_test_split(vids, test_size=test_size)
    x_train, x_val = train_test_split(x_train, test_size=val_size)
    #x_train, x_val, x_test = ['17', '25', '8', '5', '2', '13', '4'], ['29', '1'], ['18', '11', '28']
    
    for c in classes:
        class_dir = os.path.join(img_dir, c)
        for phase in ["train", "val", "test"]:
            target_dir = os.path.join(img_dir, phase)
            if not os.path.exists(target_dir):
                os.mkdir(target_dir)

            if phase == "train":
                target = x_train
            elif phase == "val":
                target = x_val
            elif phase == "test":
                target = x_test

            for vid in tqdm(target):
                origin_dir = os.path.join(class_dir, str(vid)+'_*')
                dst_dir = os.path.join(target_dir, os.path.basename(class_dir))
                if not os.path.exists(dst_dir):
                    os.mkdir(dst_dir)
                os.system("mv %s %s" % (origin_dir, dst_dir))
            
        os.rmdir(class_dir)
    return

In [ ]:
split_train_val_test(img_dir, vids)